### Rank Text Content by Semantic Similarity

In [1]:
import pandas as pd
import numpy as np
import sys
import csv
import re
from collections import Counter
maxInt = sys.maxsize

while True:
    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [2]:
df_extracted = pd.read_csv('df_extracted.csv', engine='python', encoding='utf-8', error_bad_lines=False)

In [3]:
df_extracted.describe()

,Unnamed: 0,count
count,18977.000000,1.897700e+04
mean,11927.675344,9.184705e+04
std,6883.190923,1.599096e+05
min,0.000000,1.010000e+02
25%,5946.000000,1.025200e+04
50%,11961.000000,2.830500e+04
75%,17885.000000,1.055390e+05
max,23795.000000,2.708538e+06


In [4]:
df_extracted['Point'] = ""
df_extracted['Percent'] = ""
df_extracted['TFIDF'] = ""

In [5]:
df_extracted_sample = df_extracted[:5]
df_extracted_sample['Point'] = ""
df_extracted_sample['Percent'] = ""
df_extracted_sample['TFIDF'] = ""
df_extracted_sample

/Users/ohikendoit/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/ohikendoit/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ohikendoit/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

,Unnamed: 0,file_name,clean_text,count,Point,Percent,TFIDF
0,0,OLVEA - OLVEAct Now Report - EN - 2019-20.pdf.txt,CORPORATE SOCIAL RESPONSIBILITY PERFORMANCE RE...,29122,,,
1,1,20150416_COP_N°5_EN.pdf.txt,Sustainable development the energy that fuels ...,11939,,,
2,2,2017_report_-_FINAL__UNLOCKED.pdf.txt,The whole or any part of this work may not be ...,226975,,,
3,3,COP_Report_SMART_-_Apr_2013_final.pdf.txt,PTSinarMasAgro Resources andTechnology TbkGlob...,37025,,,
4,4,cop_ETS_2019.pdf.txt,Eser Contracting and Industry Co. Inc. Communi...,35188,,,


In [6]:
!pip install scikit-learn
!pip install gensim
!pip install nltk

*Document:* a piece of text, in the form of a string. This could be just a few words, or a whole novel.  
*Corpus:* a collection of documents.  
*Term:* a word in a document.

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords

In [8]:
# Download stopwords list
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ohikendoit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ohikendoit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ohikendoit/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
# Interface lemma tokenizer from nltk with sklearn
class LemmaTokenizer:
    ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`']
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if t not in self.ignore_tokens]

In [10]:
# Lemmatize the stop words
tokenizer = LemmaTokenizer()
token_stop = tokenizer(' '.join(stop_words))

In [11]:
search_term = ['conflict', 'violence', 'displacement', 'drought', 'earthquake', 'fire', 'flooding', 'freeze', 'health emergency', 'dengue', 'pneumonic plague', 'measles', 'landslide', 'tropical storm', 'typhoon', 'cyclone', 'hurricane', 'tsunami', 'urban disaster', 'volcanic eruption', 'refugee', 'terrorist attack', 'cold wave', 'complex emergency', 'epidemic', 'extratropical cyclone', 'flash flood', 'flood', 'heat wave', 'insect infestation', 'land slide', 'mud slide','snow avalanche', 'storm surge', 'technological disaster', 'tropical cyclone', 'volcano', 'wild fire', 'flood crisis', 'explosion', 'affected tornado', 'affected', 'death toll', 'tornado relief', 'flood appeal', 'massive explosion', 'affected areas', 'praying victims', 'injured', 'lurches fire', 'flood relief', 'flood affected', 'tornado victims', 'deadly', 'evacuated', 'relief', 'flood death', 'deaths confirmed', 'affected flooding', 'people killed', 'flood damage', 'people dead', 'major flood', 'rubble', 'another explosion', 'flood warnings', 'tornado survivor', 'damage', 'devastating', 'flood toll', 'affected hurricane', 'crisis', 'relief efforts', 'flood emergency', 'fire flood', 'huge explosion', 'bushfire', 'torrential rains', 'affected explosion', 'disaster', 'twister', 'blast', 'injuries reported', 'fatalities', 'large explosion', 'destroyed', 'displaced', 'casualties', 'climate change', 'major explosion', 'response disasters', 'explosion victims', 'tragic', 'dealing hurricane', 'flood recovery', 'dead torrential', 'flood', 'massive tornado', 'buried alive', 'alive rubble', 'crisis rises', 'flood ravaged', 'killed injured', 'killed people', 'people died', 'floods kill', 'tornado damage', 'facing flood', 'deadly explosion', 'flood disaster', 'tornado disaster', 'help victims', 'hundreds homes', 'severe flooding', 'magnitude', 'firefighters police', 'fire explosion', 'storm', 'flood hits', 'floodwaters', 'emergency', 'flood alerts', 'crisis unfolds', 'tragic events', 'deadly tornado', 'people trapped', 'surging floods', 'city tornado', 'damaged hurricane', 'rains severely', 'house flood', 'devastating tornado', 'lost lives', 'reportedly dead', 'following explosion', 'tornado flood', 'early warninig', 'warning', 'dead floods', 'flood threat', 'flood situation', 'risk running', 'loss life', 'thoughts victims', 'terrible explosion', 'seismic', 'flood homeowners', 'flood claims', 'power supplies', 'free hotline', 'hotline help', 'registered magnitude', 'prepare hurricane', 'landfall', 'crisis worsens', 'communities damaged', 'destruction', 'tornado', 'hurricane coming', 'toxins flood', 'release toxins', 'toxins', 'supplies waters', 'crisis found', 'braces major', 'government negligent', 'attack', 'waiting hurricane', 'terror', 'memorial service', 'terror attack', 'coast hurricane', 'terrified hurricane', 'hurricane category', 'disaster relief', 'cleanup', 'troops lend', 'effected hurricane', 'time hurricane', 'saying hurricane', 'praying families', 'dramatic', 'path hurricane']
search_term = list(dict.fromkeys(search_term))

search_term_words = 'conflict violence displacement drought earthquake fire flooding freeze health emergency dengue pneumonic plague measles landslide tropical storm typhoon cyclone hurricane tsunami urban disaster volcanic eruption refugee terrorist attack cold wave complex emergency epidemic extratropical cyclone flash flood flood heat wave insect infestation land slide mud slidesnow avalanche storm surge technological disaster tropical cyclone volcano wild fire flood crisis explosion affected tornado affected death toll tornado relief flood appeal massive explosion affected areas praying victims injured lurches fire flood relief flood affected tornado victims deadly evacuated relief flood death deaths confirmed affected flooding people killed flood damage people dead major flood rubble another explosion flood warnings tornado survivor damage devastating flood toll affected hurricane crisis relief efforts flood emergency fire flood huge explosion bushfire torrential rains affected explosion disaster twister blast injuries reported fatalities large explosion destroyed displaced casualties climate change major explosion response disasters explosion victims tragic dealing hurricane flood recovery dead torrential flood massive tornado buried alive alive rubble crisis rises flood ravaged killed injured killed people people died floods kill tornado damage facing flood deadly explosion flood disaster tornado disaster help victims hundreds homes severe flooding magnitude firefighters police fire explosion storm flood hits floodwaters emergency flood alerts crisis unfolds tragic events deadly tornado people trapped surging floods city tornado damaged hurricane rains severely house flood devastating tornado lost lives reportedly dead following explosion tornado flood early warninig warning dead floods flood threat flood situation risk running loss life thoughts victims terrible explosion seismic flood homeowners flood claims power supplies free hotline hotline help registered magnitude prepare hurricane landfall crisis worsens communities damaged destruction tornado hurricane coming toxins flood release toxins toxins supplies waters crisis found braces major government negligent attack waiting hurricane terror memorial service terror attack coast hurricane terrified hurricane hurricane category disaster relief cleanup troops lend effected hurricane time hurricane saying hurricane praying families dramatic path hurricane'
#search_term_words = search_term_words.values.tolist()

#search_term_dic = {i: search_term[i] for i in range(len(search_term))}
#search_term = search_term_dic

In [12]:
sample = df_extracted_sample['clean_text'][1]

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(vocabulary=search_term, lowercase=True, stop_words = 'english', ngram_range = (1, 4),
                             max_df = 100, min_df = 1)
count_vectorizer.fit(search_term)
count_vectorizer.vocabulary_

{'conflict': 0,
 'violence': 1,
 'displacement': 2,
 'drought': 3,
 'earthquake': 4,
 'fire': 5,
 'flooding': 6,
 'freeze': 7,
 'health emergency': 8,
 'dengue': 9,
 'pneumonic plague': 10,
 'measles': 11,
 'landslide': 12,
 'tropical storm': 13,
 'typhoon': 14,
 'cyclone': 15,
 'hurricane': 16,
 'tsunami': 17,
 'urban disaster': 18,
 'volcanic eruption': 19,
 'refugee': 20,
 'terrorist attack': 21,
 'cold wave': 22,
 'complex emergency': 23,
 'epidemic': 24,
 'extratropical cyclone': 25,
 'flash flood': 26,
 'flood': 27,
 'heat wave': 28,
 'insect infestation': 29,
 'land slide': 30,
 'mud slide': 31,
 'snow avalanche': 32,
 'storm surge': 33,
 'technological disaster': 34,
 'tropical cyclone': 35,
 'volcano': 36,
 'wild fire': 37,
 'flood crisis': 38,
 'explosion': 39,
 'affected tornado': 40,
 'affected': 41,
 'death toll': 42,
 'tornado relief': 43,
 'flood appeal': 44,
 'massive explosion': 45,
 'affected areas': 46,
 'praying victims': 47,
 'injured': 48,
 'lurches fire': 49,
 'f

In [14]:
#Method 1: Sum of occurance to a point
X = count_vectorizer.fit_transform([sample])
vocab = count_vectorizer.get_feature_names()
counts = X.sum(axis=0).A1
freq_distribution = Counter(dict(zip(vocab, counts)))

allwords = dict(freq_distribution)
totalnum = sum(allwords.values())

In [15]:
#Method 1 - Implementation
for i in range(len(df_extracted['clean_text'])):
    clean_text = df_extracted['clean_text'][i]
    point = df_extracted['Point'][i]
    
    X = count_vectorizer.fit_transform([clean_text])
    counts = X.sum(axis=0).A1
    freq_distribution = Counter(dict(zip(vocab, counts)))
    
    allwords = dict(freq_distribution)
    totalnum = sum(allwords.values())
    df_extracted['Point'][i] = totalnum

/Users/ohikendoit/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [16]:
df_extracted

,Unnamed: 0,file_name,clean_text,count,Point,Percent,TFIDF
0,0,OLVEA - OLVEAct Now Report - EN - 2019-20.pdf.txt,CORPORATE SOCIAL RESPONSIBILITY PERFORMANCE RE...,29122,5,,
1,1,20150416_COP_N°5_EN.pdf.txt,Sustainable development the energy that fuels ...,11939,0,,
2,2,2017_report_-_FINAL__UNLOCKED.pdf.txt,The whole or any part of this work may not be ...,226975,1,,
3,3,COP_Report_SMART_-_Apr_2013_final.pdf.txt,PTSinarMasAgro Resources andTechnology TbkGlob...,37025,1,,
4,4,cop_ETS_2019.pdf.txt,Eser Contracting and Industry Co. Inc. Communi...,35188,2,,
...,...,...,...,...,...,...,...
18972,23789,CSR_Report_2014.pdf.txt,AH Industries Corporate Social R esponsibility...,39306,2,,
18973,23792,Nilorngruppen_Sustainability_report.pdf.txt,1More fit for the future SUSTAINABILITY REPORT...,112118,13,,
18974,23793,MM_CSR_2018.pdf.txt,ANNUAL CORPORATE SOCIAL RESPONSIBILITY REPORT ...,39360,3,,
18975,23794,AmatheonAgri-COP-2017.pdf.txt,01AMATHEON AGRI COMMUNICATION ON PROGRESS 2017...,21804,4,,


In [17]:
print(totalnum)

0


In [18]:
#Method 2: Percentage in relevance
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

compare = [sample, search_term_words]
cVect = CountVectorizer()
cMatrix = cVect.fit_transform(compare)

matPercent = cosine_similarity(cMatrix)[0][1] * 100
matPercent = round(matPercent, 2)
print("Report is a  "+ str(matPercent)+ "% relvant to disaster")

Report is a  0.21% relvant to disaster


In [19]:
#Method 2 - Implementation
for i in range(len(df_extracted['clean_text'])):
    clean_text = df_extracted['clean_text'][i]
    point = df_extracted['Percent'][i]
    
    compare = [clean_text, search_term_words]
    cVect = CountVectorizer()
    cMatrix = cVect.fit_transform(compare)

    matPercent = cosine_similarity(cMatrix)[0][1] * 100
    matPercent = round(matPercent, 2)
    
    df_extracted['Percent'][i] = str(matPercent)+"%"

/Users/ohikendoit/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [20]:
df_extracted

,Unnamed: 0,file_name,clean_text,count,Point,Percent,TFIDF
0,0,OLVEA - OLVEAct Now Report - EN - 2019-20.pdf.txt,CORPORATE SOCIAL RESPONSIBILITY PERFORMANCE RE...,29122,5,0.67%,
1,1,20150416_COP_N°5_EN.pdf.txt,Sustainable development the energy that fuels ...,11939,0,0.21%,
2,2,2017_report_-_FINAL__UNLOCKED.pdf.txt,The whole or any part of this work may not be ...,226975,1,0.4%,
3,3,COP_Report_SMART_-_Apr_2013_final.pdf.txt,PTSinarMasAgro Resources andTechnology TbkGlob...,37025,1,0.51%,
4,4,cop_ETS_2019.pdf.txt,Eser Contracting and Industry Co. Inc. Communi...,35188,2,0.39%,
...,...,...,...,...,...,...,...
18972,23789,CSR_Report_2014.pdf.txt,AH Industries Corporate Social R esponsibility...,39306,2,0.65%,
18973,23792,Nilorngruppen_Sustainability_report.pdf.txt,1More fit for the future SUSTAINABILITY REPORT...,112118,13,0.43%,
18974,23793,MM_CSR_2018.pdf.txt,ANNUAL CORPORATE SOCIAL RESPONSIBILITY REPORT ...,39360,3,0.91%,
18975,23794,AmatheonAgri-COP-2017.pdf.txt,01AMATHEON AGRI COMMUNICATION ON PROGRESS 2017...,21804,4,0.55%,


In [21]:
features = count_vectorizer.get_feature_names()
for feature in features:
    print (feature)

conflict
violence
displacement
drought
earthquake
fire
flooding
freeze
health emergency
dengue
pneumonic plague
measles
landslide
tropical storm
typhoon
cyclone
hurricane
tsunami
urban disaster
volcanic eruption
refugee
terrorist attack
cold wave
complex emergency
epidemic
extratropical cyclone
flash flood
flood
heat wave
insect infestation
land slide
mud slide
snow avalanche
storm surge
technological disaster
tropical cyclone
volcano
wild fire
flood crisis
explosion
affected tornado
affected
death toll
tornado relief
flood appeal
massive explosion
affected areas
praying victims
injured
lurches fire
flood relief
flood affected
tornado victims
deadly
evacuated
relief
flood death
deaths confirmed
affected flooding
people killed
flood damage
people dead
major flood
rubble
another explosion
flood warnings
tornado survivor
damage
devastating
flood toll
affected hurricane
crisis
relief efforts
flood emergency
fire flood
huge explosion
bushfire
torrential rains
affected explosion
disaster
twi

In [22]:
vect = count_vectorizer.transform([sample])
v0 = count_vectorizer.transform([sample]).toarray()
print(np.sum(v0))
print(vect)

0



In [23]:
#Method 3 TF-IDF value

vectorizer = TfidfVectorizer(vocabulary=search_term, stop_words = 'english', ngram_range = (1, 4),
                             max_df = 100, min_df = 1, max_features = 100000)
vectorizer.fit([sample])

#df_keywords = pd.DataFrame(vectorizer.transform(search_term.values()).todense(),
#                          index = search_term.keys(),
#                          columns = vectorizer.get_feature_names())
#df_keywords.index.name = 'report'

#print('Shape:', df_keywords.shape)
#display(df_keywords.head())

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=100,
                max_features=100000, min_df=1, ngram_range=(1, 4), norm='l2',
                preprocessor=None, smooth_idf=True, stop_words='english',
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=N...
                vocabulary=['conflict', 'violence', 'displacement', 'drought',
                            'earthquake', 'fire', 'flooding', 'freeze',
                            'health emergency', 'dengue', 'pneumonic plague',
                            'measles', 'landslide', 'tropical storm', 'typhoon',
                            'cyclone', 'hurricane', 'tsunami', 'urban disaster',
                            'volcanic eruption', 'refugee', 'terrorist attack',
                            'cold wave', 'complex emergency

In [24]:
X = vectorizer.fit_transform([sample])
vocab = vectorizer.get_feature_names()
counts = X.sum(axis=0).A1
freq_distribution = Counter(dict(zip(vocab, counts)))

allwords = dict(freq_distribution)
totalnum = sum(allwords.values())

for i in range(len(df_extracted_sample['clean_text'])):
    clean_text = df_extracted_sample['clean_text'][i]
    point = df_extracted_sample['Point'][i]
    
    X = vectorizer.fit_transform([clean_text])
    counts = X.sum(axis=0).A1
    freq_distribution = Counter(dict(zip(vocab, counts)))
    
    allwords = dict(freq_distribution)
    totalnum = sum(allwords.values())
    df_extracted_sample['Point'][i] = totalnum

In [25]:
totalnum

0.0

In [26]:
#Method 3 - Implementation TF-IDF value

for i in range(len(df_extracted['clean_text'])):
    clean_text = df_extracted['clean_text'][i]
    
    X = vectorizer.fit_transform([clean_text])
    counts = X.sum(axis=0).A1
    freq_distribution = Counter(dict(zip(vocab, counts)))
    
    allwords = dict(freq_distribution)
    totalnum = sum(allwords.values())
    df_extracted['TFIDF'][i] = totalnum

/Users/ohikendoit/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [27]:
df_extracted

,Unnamed: 0,file_name,clean_text,count,Point,Percent,TFIDF
0,0,OLVEA - OLVEAct Now Report - EN - 2019-20.pdf.txt,CORPORATE SOCIAL RESPONSIBILITY PERFORMANCE RE...,29122,5,0.67%,1.507557
1,1,20150416_COP_N°5_EN.pdf.txt,Sustainable development the energy that fuels ...,11939,0,0.21%,0.0
2,2,2017_report_-_FINAL__UNLOCKED.pdf.txt,The whole or any part of this work may not be ...,226975,1,0.4%,1.0
3,3,COP_Report_SMART_-_Apr_2013_final.pdf.txt,PTSinarMasAgro Resources andTechnology TbkGlob...,37025,1,0.51%,1.0
4,4,cop_ETS_2019.pdf.txt,Eser Contracting and Industry Co. Inc. Communi...,35188,2,0.39%,1.414214
...,...,...,...,...,...,...,...
18972,23789,CSR_Report_2014.pdf.txt,AH Industries Corporate Social R esponsibility...,39306,2,0.65%,1.414214
18973,23792,Nilorngruppen_Sustainability_report.pdf.txt,1More fit for the future SUSTAINABILITY REPORT...,112118,13,0.43%,1.857143
18974,23793,MM_CSR_2018.pdf.txt,ANNUAL CORPORATE SOCIAL RESPONSIBILITY REPORT ...,39360,3,0.91%,1.341641
18975,23794,AmatheonAgri-COP-2017.pdf.txt,01AMATHEON AGRI COMMUNICATION ON PROGRESS 2017...,21804,4,0.55%,1.632993


In [28]:
#Finalization
df_extracted.to_csv('df_extracted_ranked.csv')

In [29]:
pd.read_csv('df_extracted_ranked.csv', engine='python', encoding='utf-8', error_bad_lines=False)

,Unnamed: 0,Unnamed: 0.1,file_name,clean_text,count,Point,Percent,TFIDF
0,0,0,OLVEA - OLVEAct Now Report - EN - 2019-20.pdf.txt,CORPORATE SOCIAL RESPONSIBILITY PERFORMANCE RE...,29122,5,0.67%,1.507557
1,1,1,20150416_COP_N°5_EN.pdf.txt,Sustainable development the energy that fuels ...,11939,0,0.21%,0.000000
2,2,2,2017_report_-_FINAL__UNLOCKED.pdf.txt,The whole or any part of this work may not be ...,226975,1,0.4%,1.000000
3,3,3,COP_Report_SMART_-_Apr_2013_final.pdf.txt,PTSinarMasAgro Resources andTechnology TbkGlob...,37025,1,0.51%,1.000000
4,4,4,cop_ETS_2019.pdf.txt,Eser Contracting and Industry Co. Inc. Communi...,35188,2,0.39%,1.414214
...,...,...,...,...,...,...,...,...
18972,18972,23789,CSR_Report_2014.pdf.txt,AH Industries Corporate Social R esponsibility...,39306,2,0.65%,1.414214
18973,18973,23792,Nilorngruppen_Sustainability_report.pdf.txt,1More fit for the future SUSTAINABILITY REPORT...,112118,13,0.43%,1.857143
18974,18974,23793,MM_CSR_2018.pdf.txt,ANNUAL CORPORATE SOCIAL RESPONSIBILITY REPORT ...,39360,3,0.91%,1.341641
18975,18975,23794,AmatheonAgri-COP-2017.pdf.txt,01AMATHEON AGRI COMMUNICATION ON PROGRESS 2017...,21804,4,0.55%,1.632993
